In [3]:
import os
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
#from google.colab import files
from tensorflow.keras.preprocessing import image
import random
from   tensorflow.keras.preprocessing.image import img_to_array, load_img


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [1]:
#os.listdir("../DroneTrainerNotebooks/test/ok")
#!ls  -l ../DroneTrainerNotebooks/data/ok
#!file  ../DroneTrainerNotebooks/data/ok/*

In [5]:
#base_dir = '/test'
train_dir = 'data'
validation_dir = 'test'

train_ok_dir = os.path.join(train_dir, 'ok')
train_ko_dir = os.path.join(train_dir, 'ko')

validation_ok_dir = os.path.join(validation_dir, 'ok')
validation_ko_dir = os.path.join(validation_dir, 'ko')

train_ok_fnames = os.listdir( train_ok_dir )
train_ko_fnames = os.listdir( train_ko_dir )

print('total training ok images :', len(os.listdir(      train_ok_dir ) ))
print('total training ko images :', len(os.listdir(      train_ko_dir ) ))

print('total validation ok images :', len(os.listdir( validation_ok_dir ) ))
print('total validation ko images :', len(os.listdir( validation_ko_dir ) ))

total training ok images : 65
total training ko images : 160
total validation ok images : 14
total validation ko images : 31


In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(160, 120, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(512, activation='relu'), 
    tf.keras.layers.Dense(1, activation='sigmoid')  
])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 158, 118, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 79, 59, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 77, 57, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 38, 28, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 36, 26, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 13, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 14976)             0

In [7]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics = ['acc'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=2,
                                                    class_mode='binary',
                                                    target_size=(160, 120))     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=2,
                                                         class_mode  = 'binary',
                                                         target_size = (160, 120))


Found 225 images belonging to 2 classes.
Found 45 images belonging to 2 classes.


In [21]:
history = model.fit_generator(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=100,
                              epochs=20,
                              validation_steps=20,
                              verbose=2)

Epoch 1/20
100/100 - 8s - loss: 0.0485 - acc: 0.9950 - val_loss: 24.4270 - val_acc: 0.7000
Epoch 2/20
100/100 - 8s - loss: 0.1898 - acc: 0.9950 - val_loss: 2.5272 - val_acc: 0.8000
Epoch 3/20
100/100 - 8s - loss: 0.0532 - acc: 0.9900 - val_loss: 0.7565 - val_acc: 0.8250
Epoch 4/20
100/100 - 8s - loss: 7.6541e-04 - acc: 1.0000 - val_loss: 9.9927 - val_acc: 0.7000
Epoch 5/20
100/100 - 8s - loss: 0.2611 - acc: 0.9850 - val_loss: 1.4386 - val_acc: 0.8250
Epoch 6/20
100/100 - 8s - loss: 0.1728 - acc: 0.9899 - val_loss: 0.5195 - val_acc: 0.8750
Epoch 7/20
100/100 - 8s - loss: 9.5454e-04 - acc: 1.0000 - val_loss: 2.8524 - val_acc: 0.7750
Epoch 8/20
100/100 - 8s - loss: 7.9608e-07 - acc: 1.0000 - val_loss: 2.2610 - val_acc: 0.8000
Epoch 9/20
100/100 - 8s - loss: 1.5967e-08 - acc: 1.0000 - val_loss: 3.0454 - val_acc: 0.7750
Epoch 10/20
100/100 - 8s - loss: 0.0083 - acc: 0.9950 - val_loss: 1.2942 - val_acc: 0.8250
Epoch 11/20
100/100 - 8s - loss: 3.5046e-07 - acc: 1.0000 - val_loss: 1.4881 - val

In [20]:

#uploaded=files.upload()

#for fn in uploaded.keys():
for i in os.listdir('immaginiprova'):
 
  # predicting images
  path='./immaginiprova/' + i
  img=image.load_img(path, target_size=(150, 150))
  
  x=image.img_to_array(img)
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])
  
  classes = model.predict(images, batch_size=10)
  
  print(classes[0])
  
  if classes[0]>0:
    print(i + " is ok")
    
  else:
    print(i + " is ko")
 

ValueError: Error when checking input: expected conv2d_input to have shape (160, 120, 3) but got array with shape (150, 150, 3)

In [ ]:
os.listdir('immaginiprova')
for i in os.listdir('immaginiprova'):
    print(i)
    

In [ ]:
successive_outputs = [layer.output for layer in model.layers[1:]]
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)
ok_img_files = [os.path.join(train_ok_dir, f) for f in train_ok_fnames]
ko_img_files = [os.path.join(train_ko_dir, f) for f in train_ko_fnames]
img_path = random.choice(ok_img_files + ko_img_files)
img = load_img(img_path, target_size=(150, 150))  # this is a PIL image

x   = img_to_array(img)                           # Numpy array with shape (150, 150, 3)
x   = x.reshape((1,) + x.shape)                   # Numpy array with shape (1, 150, 150, 3)

# Rescale by 1/255
x /= 255.0

# Let's run our image through our network, thus obtaining all
# intermediate representations for this image.
successive_feature_maps = visualization_model.predict(x)

# These are the names of the layers, so can have them as part of our plot
layer_names = [layer.name for layer in model.layers]

# -----------------------------------------------------------------------
# Now let's display our representations
# -----------------------------------------------------------------------
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  
  if len(feature_map.shape) == 4:
    
    #-------------------------------------------
    # Just do this for the conv / maxpool layers, not the fully-connected layers
    #-------------------------------------------
    n_features = feature_map.shape[-1]  # number of features in the feature map
    size       = feature_map.shape[ 1]  # feature map shape (1, size, size, n_features)
    
    # We will tile our images in this matrix
    display_grid = np.zeros((size, size * n_features))
    
    #-------------------------------------------------
    # Postprocess the feature to be visually palatable
    #-------------------------------------------------
    for i in range(n_features):
      x  = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std ()
      x *=  64
      x += 128
      x  = np.clip(x, 0, 255).astype('uint8')
      display_grid[:, i * size : (i + 1) * size] = x # Tile each filter into a horizontal grid

    #-----------------
    # Display the grid
    #-----------------

    scale = 20. / n_features
    plt.figure( figsize=(scale * n_features, scale) )
    plt.title ( layer_name )
    plt.grid  ( False )
    plt.imshow( display_grid, aspect='auto', cmap='viridis' ) 

In [ ]:
#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc      = history.history[     'acc' ]
val_acc  = history.history[ 'val_acc' ]
loss     = history.history[    'loss' ]
val_loss = history.history['val_loss' ]

epochs   = range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot  ( epochs,     acc )
plt.plot  ( epochs, val_acc )
plt.title ('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot  ( epochs,     loss )
plt.plot  ( epochs, val_loss )
plt.title ('Training and validation loss'   )

In [ ]:
import os, signal

os.kill(     os.getpid() , 
         signal.SIGKILL
       )